In [4]:
# ===== 0) Install dependencies =====
!pip -q install yfinance fastparquet

# ===== 1) Mount Google Drive =====
from google.colab import drive
drive.mount('/content/drive')

# ===== 2) Folder structure =====
from pathlib import Path
PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")
PRICES_DIR   = PROJECT_ROOT / "data" / "prices"
QRET_DIR     = PROJECT_ROOT / "data" / "qreturns"
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)
PRICES_DIR.mkdir(parents=True, exist_ok=True)
QRET_DIR.mkdir(parents=True, exist_ok=True)

# ===== 3) Fixed Tech 20 tickers =====
tickers = [
    "AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA",
    "AVGO","AMD","INTC","QCOM","TXN","MU",
    "CRM","ORCL","ADBE",
    "IBM","HPQ","DELL","CSCO"
]
# Adjust for Yahoo convention
tickers = [t.replace(".", "-") for t in tickers]
print("Universe size:", len(tickers))
print(tickers)

# ===== 4) Download and cache daily prices =====
import yfinance as yf
import pandas as pd, time, sys

start, end = "2010-01-01", "2024-01-01"

for t in tickers:
    fpath = PRICES_DIR/f"{t}.parquet"
    if fpath.exists():
        continue
    try:
        print(f"Downloading {t}...")
        df = yf.download(t, start=start, end=end, auto_adjust=False, progress=False).reset_index()
        # Ensure Adj Close column exists
        if 'Adj Close' not in df.columns:
            df = yf.download(t, start=start, end=end, auto_adjust=True, progress=False).reset_index()
            df = df.rename(columns={'Close':'Adj Close'})
        df.to_parquet(fpath)
        time.sleep(1)  # throttle
    except Exception as e:
        print(f"Failed {t}: {e}", file=sys.stderr)

print("Price cache complete.")

# ===== 5) Convert to quarterly returns =====
for t in tickers:
    fp = PRICES_DIR/f"{t}.parquet"
    if not fp.exists():
        continue
    df = pd.read_parquet(fp)
    if 'Date' not in df.columns or 'Adj Close' not in df.columns:
        continue
    px = (df[['Date','Adj Close']].dropna()
            .assign(Date=lambda d: pd.to_datetime(d['Date']))
            .set_index('Date').sort_index()['Adj Close'])
    px_q = px.resample('Q').last()
    qret = px_q.pct_change().dropna().to_frame('qret').reset_index()
    qret['ticker'] = t
    qret.rename(columns={'Date':'quarter_end'}, inplace=True)
    qret.to_parquet(QRET_DIR/f"{t}.parquet")

# ===== 6) Merge into one quarterly panel =====
frames = []
for t in tickers:
    fp = QRET_DIR/f"{t}.parquet"
    if fp.exists():
        frames.append(pd.read_parquet(fp))
if frames:
    panel = pd.concat(frames, ignore_index=True)
    panel.to_parquet(PROJECT_ROOT/'data'/'tech20_quarterly_returns.parquet')
    print("Saved panel:", PROJECT_ROOT/'data'/'tech20_quarterly_returns.parquet')
    print(panel.head())
else:
    print("No quarterly returns generated.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Universe size: 20
['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'AVGO', 'AMD', 'INTC', 'QCOM', 'TXN', 'MU', 'CRM', 'ORCL', 'ADBE', 'IBM', 'HPQ', 'DELL', 'CSCO']
Price cache complete.


/tmp/ipython-input-3799846647.py:64: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  px_q = px.resample('Q').last()


AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [5]:
# ===== Convert cached daily prices to quarterly returns =====
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")
PRICES_DIR   = PROJECT_ROOT / "data" / "prices"
QRET_DIR     = PROJECT_ROOT / "data" / "qreturns"
QRET_DIR.mkdir(parents=True, exist_ok=True)

tickers = [
    "AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA",
    "AVGO","AMD","INTC","QCOM","TXN","MU",
    "CRM","ORCL","ADBE","IBM","HPQ","DELL","CSCO"
]

for t in tickers:
    fp = PRICES_DIR/f"{t}.parquet"
    if not fp.exists():
        continue
    df = pd.read_parquet(fp)
    if 'Date' not in df.columns or 'Adj Close' not in df.columns:
        continue

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    px = df.set_index('Date')['Adj Close'].dropna()

    # Quarterly frequency with quarter-end
    px_q = px.resample('QE').last()
    qret = px_q.pct_change().dropna().reset_index()
    qret.columns = ['quarter_end', 'qret']
    qret['ticker'] = t

    qret.to_parquet(QRET_DIR/f"{t}.parquet")

# ===== Merge into one panel =====
frames = []
for t in tickers:
    fp = QRET_DIR/f"{t}.parquet"
    if fp.exists():
        frames.append(pd.read_parquet(fp))

if frames:
    panel = pd.concat(frames, ignore_index=True)
    panel.to_parquet(PROJECT_ROOT/'data'/'tech20_quarterly_returns.parquet')
    print("Saved:", PROJECT_ROOT/'data'/'tech20_quarterly_returns.parquet')
    print(panel.head())
else:
    print("No quarterly returns generated.")


Saved: /content/drive/MyDrive/alpha_project/data/tech20_quarterly_returns.parquet
  quarter_end      qret ticker
0  2010-06-30  0.070340   AAPL
1  2010-09-30  0.128096   AAPL
2  2010-12-31  0.136776   AAPL
3  2011-03-31  0.080450   AAPL
4  2011-06-30 -0.036843   AAPL


In [9]:
# ===== Load datasets =====
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")

indicators = pd.read_csv("/content/indicators_by_company.csv")
companies  = pd.read_csv("/content/companies.csv")

# ===== Tech 20 tickers and stricter patterns (corrected META, HPQ) =====
ticker_map = {
    "AAPL": "^APPLE INC",
    "MSFT": "^MICROSOFT",
    "AMZN": "^AMAZON",
    "GOOGL": "ALPHABET|GOOGLE",
    "META": "FACEBOOK|META PLATFORMS",
    "TSLA": "^TESLA",
    "NVDA": "^NVIDIA",
    "AVGO": "^BROADCOM",
    "AMD": "^ADVANCED MICRO",
    "INTC": "^INTEL CORP",
    "QCOM": "^QUALCOMM",
    "TXN": "^TEXAS INSTRUMENTS",
    "MU": "^MICRON",
    "CRM": "^SALESFORCE",
    "ORCL": "^ORACLE",
    "ADBE": "^ADOBE",
    "IBM": "^INTERNATIONAL BUSINESS MACHINES",
    "HPQ": "^HP INC",       # force HP Inc (not Hewlett Packard Enterprise)
    "DELL": "^DELL",
    "CSCO": "^CISCO SYSTEMS"
}

# ===== Step 1: Match tickers to company_id =====
companies['name_upper'] = companies['name_latest'].str.upper()

matches = []
for t, pattern in ticker_map.items():
    hit = companies[companies['name_upper'].str.contains(pattern, regex=True, na=False)]
    if not hit.empty:
        # Filter out junk like REIT, TRUST, BANK
        hit = hit[~hit['name_upper'].str.contains("TRUST|REIT|BANK", na=False)]
        if not hit.empty:
            row = hit.iloc[0]  # take first valid match
            matches.append({"ticker": t, "company_id": row['company_id'], "name": row['name_latest']})
        else:
            print("Filtered out junk for", t)
    else:
        print("No match for", t)

mapping_df = pd.DataFrame(matches)
print("Ticker mapping (cleaned):")
print(mapping_df)

# ===== Step 2: Filter fundamentals for tech company_ids =====
tech_ids = mapping_df['company_id'].tolist()
funds = indicators[indicators['company_id'].isin(tech_ids)].copy()

# Melt to tidy format
value_cols = [c for c in funds.columns if c.isdigit()]
funds_tidy = funds.melt(id_vars=['company_id','indicator_id'],
                        value_vars=value_cols,
                        var_name='year',
                        value_name='value')
funds_tidy['year'] = funds_tidy['year'].astype(int)

# Pivot indicators to wide format
funds_wide = funds_tidy.pivot_table(index=['company_id','year'],
                                    columns='indicator_id',
                                    values='value')
funds_wide = funds_wide.reset_index()

# ===== Step 3: Attach tickers =====
funds_wide = funds_wide.merge(mapping_df[['ticker','company_id']],
                              on='company_id', how='inner')

print("Fundamentals panel shape:", funds_wide.shape)
print(funds_wide[['ticker','year']].head())


Ticker mapping (cleaned):
   ticker  company_id                                  name
0    AAPL      320193                             Apple Inc
1    MSFT      789019                        Microsoft Corp
2    AMZN     1018724                        Amazon COM Inc
3   GOOGL     1288776                           Google Inc.
4    META     1326801                          Facebook Inc
5    TSLA     1318605                      Tesla Motors Inc
6    NVDA     1045810                           Nvidia Corp
7    AVGO     1054374                         Broadcom Corp
8     AMD        2488            Advanced Micro Devices Inc
9    INTC       50863                            Intel Corp
10   QCOM      804328                       Qualcomm Inc/De
11    TXN       97476                 Texas Instruments Inc
12     MU      723125                 Micron Technology Inc
13    CRM     1108524                    Salesforce COM Inc
14   ORCL     1341439                           Oracle Corp
15   ADBE     

In [11]:
# ===== Scan indicator IDs by keyword and save to CSV =====
import pandas as pd

indicators = pd.read_csv("/content/indicators_by_company.csv")

unique_ids = indicators['indicator_id'].unique().tolist()

keywords = ["NETINCOME", "ASSET", "LIABILIT", "EQUITY",
            "REVENUE", "SALES", "GROSSPROFIT", "SHARES"]

rows = []
for kw in keywords:
    hits = [x for x in unique_ids if kw in x.upper()]
    for h in hits:
        rows.append({"keyword": kw, "indicator_id": h})

ref_df = pd.DataFrame(rows)

# Save reference file
outpath = "/content/indicator_reference.csv"
ref_df.to_csv(outpath, index=False)

print("Saved reference file with matches:", outpath)
print(ref_df.groupby("keyword").size())
print(ref_df.head(20))


Saved reference file with matches: /content/indicator_reference.csv
keyword
ASSET          928
EQUITY         289
GROSSPROFIT      5
LIABILIT       715
NETINCOME       78
REVENUE        259
SALES          258
SHARES         210
dtype: int64
      keyword                                       indicator_id
0   NETINCOME                                      NetIncomeLoss
1   NETINCOME    NetIncomeLossAvailableToCommonStockholdersBasic
2   NETINCOME  AdjustmentsNoncashItemsToReconcileNetIncomeLos...
3   NETINCOME  AdjustmentsToReconcileNetIncomeLossToCashProvi...
4   NETINCOME  EquityMethodInvestmentSummarizedFinancialInfor...
5   NETINCOME  NetIncomeLossAttributableToNoncontrollingInterest
6   NETINCOME  NetIncomeLossAvailableToCommonStockholdersDiluted
7   NETINCOME  OtherComprehensiveIncomeLossReclassificationAd...
8   NETINCOME  OtherComprehensiveIncomeLossReclassificationAd...
9   NETINCOME  OtherComprehensiveIncomeReclassificationAdjust...
10  NETINCOME  OtherComprehensiveIncomeRecla

In [12]:
# ===== 0) Imports =====
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")

# ===== 1) Load fundamentals =====
indicators = pd.read_csv("/content/indicators_by_company.csv")

# Keep only relevant indicator IDs
chosen_indicators = [
    "NetIncomeLoss",
    "Assets",
    "Liabilities",
    "StockholdersEquity",
    "StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest",
    "Revenues",
    "SalesRevenueNet",
    "GrossProfit",
    "CommonStockSharesOutstanding"
]

funds = indicators[indicators['indicator_id'].isin(chosen_indicators)].copy()

# ===== 2) Reshape =====
value_cols = [c for c in funds.columns if c.isdigit()]
funds_tidy = funds.melt(id_vars=['company_id','indicator_id'],
                        value_vars=value_cols,
                        var_name='year',
                        value_name='value')
funds_tidy['year'] = funds_tidy['year'].astype(int)

funds_wide = funds_tidy.pivot_table(index=['company_id','year'],
                                    columns='indicator_id',
                                    values='value',
                                    aggfunc='first').reset_index()

# ===== 3) Attach tickers (mapping from earlier) =====
mapping_df = pd.DataFrame({
    "ticker": ["AAPL","MSFT","AMZN","GOOGL","META","TSLA","NVDA","AVGO","AMD","INTC",
               "QCOM","TXN","MU","CRM","ORCL","ADBE","IBM","HPQ","DELL","CSCO"],
    "company_id": [320193,789019,1018724,1288776,1326801,1318605,1045810,1054374,2488,50863,
                   804328,97476,723125,1108524,1341439,796343,51143,47217,826083,858877]
})
funds_wide = funds_wide.merge(mapping_df, on="company_id", how="inner")

# ===== 4) Load quarterly returns =====
returns = pd.read_parquet(PROJECT_ROOT/'data'/'tech20_quarterly_returns.parquet')
returns['year'] = pd.to_datetime(returns['quarter_end']).dt.year

# ===== 5) Merge annual fundamentals with quarterly returns =====
panel = returns.merge(funds_wide, on=['ticker','year'], how='left')

# ===== 6) Compute ratios =====
# Equity: pick whichever of the two equity columns is non-null
panel['Equity_clean'] = panel[['StockholdersEquity',
                               'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest']].bfill(axis=1).iloc[:,0]

panel['Revenue_clean'] = panel[['Revenues','SalesRevenueNet']].bfill(axis=1).iloc[:,0]

panel['ROE'] = panel['NetIncomeLoss'] / panel['Equity_clean']
panel['ROA'] = panel['NetIncomeLoss'] / panel['Assets']
panel['Leverage'] = panel['Liabilities'] / panel['Assets']
panel['ProfitMargin'] = panel['NetIncomeLoss'] / panel['Revenue_clean']
panel['GrossMargin'] = panel['GrossProfit'] / panel['Revenue_clean']

# ===== 7) Save final dataset =====
final_cols = ['ticker','quarter_end','qret','ROE','ROA','Leverage','ProfitMargin','GrossMargin']
panel_final = panel[final_cols]

outpath = PROJECT_ROOT/'data'/'tech20_fundamentals_returns.csv'
panel_final.to_csv(outpath, index=False)

print("Saved dataset:", outpath)
print(panel_final.head())


Saved dataset: /content/drive/MyDrive/alpha_project/data/tech20_fundamentals_returns.csv
  ticker quarter_end      qret       ROE       ROA  Leverage  ProfitMargin  \
0   AAPL  2010-06-30  0.070340       NaN       NaN       NaN           NaN   
1   AAPL  2010-09-30  0.128096       NaN       NaN       NaN           NaN   
2   AAPL  2010-12-31  0.136776       NaN       NaN       NaN           NaN   
3   AAPL  2011-03-31  0.080450  0.338341  0.222753  0.341632      0.239466   
4   AAPL  2011-06-30 -0.036843  0.338341  0.222753  0.341632      0.239466   

   GrossMargin  
0          NaN  
1          NaN  
2          NaN  
3     0.404789  
4     0.404789  


In [13]:
# ===== Preprocess fundamentals + returns for ML =====
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")

df = pd.read_csv(PROJECT_ROOT/'data'/'tech20_fundamentals_returns.csv')

# 1. Drop rows with missing qret or all-NaN predictors
predictors = ['ROE','ROA','Leverage','ProfitMargin','GrossMargin']
df = df.dropna(subset=['qret'])

# 2. Replace inf with NaN
df[predictors] = df[predictors].replace([np.inf,-np.inf], np.nan)

# 3. Drop rows with any missing predictors
df = df.dropna(subset=predictors)

# 4. Optional: winsorize (clip) to reduce effect of outliers
for col in predictors:
    lower, upper = df[col].quantile([0.01, 0.99])
    df[col] = df[col].clip(lower, upper)

# 5. Standardize predictors (z-score)
for col in predictors:
    mean, std = df[col].mean(), df[col].std()
    df[col] = (df[col] - mean) / std

# 6. Save final ML-ready dataset
outpath = PROJECT_ROOT/'data'/'tech20_ml_ready.csv'
df.to_csv(outpath, index=False)

print("Saved ML-ready dataset:", outpath)
print(df.head())


Saved ML-ready dataset: /content/drive/MyDrive/alpha_project/data/tech20_ml_ready.csv
  ticker quarter_end      qret       ROE       ROA  Leverage  ProfitMargin  \
3   AAPL  2011-03-31  0.080450  0.571551  1.349512 -1.137064      0.605392   
4   AAPL  2011-06-30 -0.036843  0.571551  1.349512 -1.137064      0.605392   
5   AAPL  2011-09-30  0.135996  0.571551  1.349512 -1.137064      0.605392   
6   AAPL  2011-12-31  0.062100  0.571551  1.349512 -1.137064      0.605392   
7   AAPL  2012-03-31  0.480370  0.597208  1.465889 -1.229378      0.699525   

   GrossMargin  
3    -0.636061  
4    -0.636061  
5    -0.636061  
6    -0.636061  
7    -0.486774  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [14]:
# ===== Load ML-ready dataset =====
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")

# Directly load the cleaned file
df = pd.read_csv(PROJECT_ROOT/'data'/'tech20_ml_ready.csv')

print("Shape:", df.shape)
print(df.head())

# Define target and predictors
target = "qret"
predictors = ['ROE','ROA','Leverage','ProfitMargin','GrossMargin']
X = df[predictors]
y = df[target]

print("Features ready for modeling:")
print(predictors)


Shape: (170, 8)
  ticker quarter_end      qret       ROE       ROA  Leverage  ProfitMargin  \
0   AAPL  2011-03-31  0.080450  0.571551  1.349512 -1.137064      0.605392   
1   AAPL  2011-06-30 -0.036843  0.571551  1.349512 -1.137064      0.605392   
2   AAPL  2011-09-30  0.135996  0.571551  1.349512 -1.137064      0.605392   
3   AAPL  2011-12-31  0.062100  0.571551  1.349512 -1.137064      0.605392   
4   AAPL  2012-03-31  0.480370  0.597208  1.465889 -1.229378      0.699525   

   GrossMargin  
0    -0.636061  
1    -0.636061  
2    -0.636061  
3    -0.636061  
4    -0.486774  
Features ready for modeling:
['ROE', 'ROA', 'Leverage', 'ProfitMargin', 'GrossMargin']


In [36]:
# ===== OLS Backtest (expanding window with forward-fill) =====
import pandas as pd
import statsmodels.api as sm
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")
df = pd.read_csv(PROJECT_ROOT/'data'/'tech20_ml_ready.csv')

factors = ['ROE','ROA','Leverage','ProfitMargin','GrossMargin']
target = 'qret'
df['quarter_end'] = pd.to_datetime(df['quarter_end'])
df = df.sort_values(['quarter_end','ticker'])

# Forward-fill predictors by ticker to avoid NaNs
df[factors] = df.groupby('ticker')[factors].ffill()

start_test = pd.Timestamp("2014-01-01")
results = []

for q in sorted(df['quarter_end'].unique()):
    if q < start_test:
        continue

    # expanding training set: all prior quarters
    train = df[df['quarter_end'] < q]
    test  = df[df['quarter_end'] == q]
    if train.empty or test.empty:
        continue

    # Ensure no NaNs in predictors
    X_train = sm.add_constant(train[factors].dropna())
    y_train = train.loc[X_train.index, target]

    if len(X_train) <= len(factors):  # skip if too few obs
        continue

    model = sm.OLS(y_train, X_train).fit()

    # Predict current quarter
    X_test = sm.add_constant(test[factors].fillna(0))  # fill if still NaN
    test = test.copy()
    test['pred'] = model.predict(X_test)

    # Long–short portfolio
    top = test.nlargest(5, 'pred')
    bottom = test.nsmallest(5, 'pred')
    long_ret = top[target].mean()
    short_ret = bottom[target].mean()
    ls_ret = long_ret - short_ret

    results.append({
        'quarter_end': q,
        'long_ret': long_ret,
        'short_ret': short_ret,
        'ls_ret': ls_ret
    })

# Build backtest dataframe
if results:
    ols_bt = pd.DataFrame(results).set_index('quarter_end')
    ols_bt['cum_ls'] = (1 + ols_bt['ls_ret']).cumprod()

    print("Backtest results (first rows):")
    print(ols_bt.head())
    print("\nBacktest results (last rows):")
    print(ols_bt.tail())
    print("\nFinal cumulative long-short return:", ols_bt['cum_ls'].iloc[-1])

    # Save results
    outpath = PROJECT_ROOT/'data'/'ols_backtest_expanding_ffill.csv'
    ols_bt.to_csv(outpath)
    print("Saved OLS backtest:", outpath)
else:
    print("No backtest results. Try an earlier start_test date or adjust NaN handling.")


Backtest results (first rows):
             long_ret  short_ret    ls_ret    cum_ls
quarter_end                                         
2014-03-31   0.114860   0.064994  0.049866  1.049866
2014-06-30   0.160972   0.056072  0.104900  1.159997
2014-09-30   0.049888   0.019615  0.030273  1.195114
2014-12-31   0.015580   0.064866 -0.049286  1.136211
2015-03-31  -0.047312   0.054334 -0.101646  1.020720

Backtest results (last rows):
             long_ret  short_ret    ls_ret    cum_ls
quarter_end                                         
2014-12-31   0.015580   0.064866 -0.049286  1.136211
2015-03-31  -0.047312   0.054334 -0.101646  1.020720
2015-06-30   0.052575   0.001356  0.051219  1.073000
2015-09-30  -0.077866   0.034844 -0.112710  0.952062
2015-12-31   0.052204   0.154204 -0.102000  0.854952

Final cumulative long-short return: 0.8549518038383018
Saved OLS backtest: /content/drive/MyDrive/alpha_project/data/ols_backtest_expanding_ffill.csv


In [17]:
# ===== Block 2: ML-style Linear Regression with Backtesting =====
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LinearRegression

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")
df = pd.read_csv(PROJECT_ROOT/'data'/'tech20_ml_ready.csv')

# Features and target
factors = ['ROE','ROA','Leverage','ProfitMargin','GrossMargin']
target = 'qret'
df['quarter_end'] = pd.to_datetime(df['quarter_end'])

# Sort by time
df = df.sort_values(['quarter_end','ticker'])

# Backtest parameters
start_test = pd.Timestamp("2014-01-01")  # start test earlier to ensure coverage
results = []

for q in sorted(df['quarter_end'].unique()):
    if q < start_test:
        continue
    train = df[df['quarter_end'] < q]
    test  = df[df['quarter_end'] == q]
    if train.empty or test.empty:
        continue

    X_train, y_train = train[factors], train[target]
    X_test, y_test   = test[factors], test[target]

    # Fit linear regression
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict current quarter returns
    test = test.copy()
    test['pred'] = model.predict(X_test)

    # Long–short portfolio: top 5 vs bottom 5 predicted returns
    top = test.nlargest(5, 'pred')
    bottom = test.nsmallest(5, 'pred')
    long_ret = top[target].mean()
    short_ret = bottom[target].mean()
    ls_ret = long_ret - short_ret

    results.append({
        'quarter_end': q,
        'long_ret': long_ret,
        'short_ret': short_ret,
        'ls_ret': ls_ret
    })

# Build backtest dataframe
if results:
    bt = pd.DataFrame(results).set_index('quarter_end')
    bt['cum_ls'] = (1 + bt['ls_ret']).cumprod()
    print("Backtest results (first rows):")
    print(bt.head())
    print("\nFinal cumulative long-short return:", bt['cum_ls'].iloc[-1])

    # Save backtest results
    outpath = PROJECT_ROOT/'data'/'linreg_backtest.csv'
    bt.to_csv(outpath)
    print("Saved backtest results:", outpath)
else:
    print("No backtest results. Try setting an earlier start_test date.")


Backtest results (first rows):
             long_ret  short_ret    ls_ret    cum_ls
quarter_end                                         
2014-03-31   0.114860   0.064994  0.049866  1.049866
2014-06-30   0.160972   0.056072  0.104900  1.159997
2014-09-30   0.049888   0.019615  0.030273  1.195114
2014-12-31   0.015580   0.064866 -0.049286  1.136211
2015-03-31  -0.047312   0.054334 -0.101646  1.020720

Final cumulative long-short return: 0.8549518038383018
Saved backtest results: /content/drive/MyDrive/alpha_project/data/linreg_backtest.csv


In [28]:
# ===== Block 3: Gradient Boosting with Backtesting =====
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.ensemble import GradientBoostingRegressor

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")
df = pd.read_csv(PROJECT_ROOT/'data'/'tech20_ml_ready.csv')

factors = ['ROE','ROA','Leverage','ProfitMargin','GrossMargin']
target = 'qret'
df['quarter_end'] = pd.to_datetime(df['quarter_end'])

# Sort by time
df = df.sort_values(['quarter_end','ticker'])

# Backtest parameters
start_test = pd.Timestamp("2014-01-01")
results = []

for q in sorted(df['quarter_end'].unique()):
    if q < start_test:
        continue
    train = df[df['quarter_end'] < q]
    test  = df[df['quarter_end'] == q]
    if train.empty or test.empty:
        continue

    X_train, y_train = train[factors], train[target]
    X_test, y_test   = test[factors], test[target]

    # Fit Gradient Boosting
    model = GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    )
    model.fit(X_train, y_train)

    # Predict current quarter returns
    test = test.copy()
    test['pred'] = model.predict(X_test)

    # Long–short portfolio: top 5 vs bottom 5 predicted returns
    top = test.nlargest(5, 'pred')
    bottom = test.nsmallest(5, 'pred')
    long_ret = top[target].mean()
    short_ret = bottom[target].mean()
    ls_ret = long_ret - short_ret

    results.append({
        'quarter_end': q,
        'long_ret': long_ret,
        'short_ret': short_ret,
        'ls_ret': ls_ret
    })

# Build backtest dataframe
if results:
    bt = pd.DataFrame(results).set_index('quarter_end')
    bt['cum_ls'] = (1 + bt['ls_ret']).cumprod()
    print("Backtest results (first rows):")
    print(bt.head())
    print("\nFinal cumulative long-short return:", bt['cum_ls'].iloc[-1])

    # Save backtest results
    outpath = PROJECT_ROOT/'data'/'gbr_backtest.csv'
    bt.to_csv(outpath)
    print("Saved backtest results:", outpath)
else:
    print("No backtest results. Try an earlier start_test date.")


Backtest results (first rows):
             long_ret  short_ret    ls_ret    cum_ls
quarter_end                                         
2014-03-31   0.124290   0.036701  0.087589  1.087589
2014-06-30   0.137890   0.153308 -0.015419  1.070820
2014-09-30   0.043020   0.046285 -0.003265  1.067324
2014-12-31   0.019538   0.064866 -0.045328  1.018944
2015-03-31   0.024339  -0.039197  0.063536  1.083684

Final cumulative long-short return: 1.085520186965541
Saved backtest results: /content/drive/MyDrive/alpha_project/data/gbr_backtest.csv


In [38]:
# ===== Summary Statistics Table =====
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_ROOT = Path("/content/drive/MyDrive/alpha_project")

# Load results
linreg = pd.read_csv(PROJECT_ROOT/'data'/'linreg_backtest.csv')
gbr    = pd.read_csv(PROJECT_ROOT/'data'/'gbr_backtest.csv')

# Try loading OLS, but handle if missing
try:
    ols_bt = pd.read_csv(PROJECT_ROOT/'data'/'ols_backtest_expanding_ffill.csv')
    ols_bt['model'] = 'OLS'
except FileNotFoundError:
    ols_bt = None

linreg['model'] = 'Linear Regression'
gbr['model'] = 'Gradient Boosting'

# Collect
dfs = []
if ols_bt is not None and not ols_bt.empty:
    dfs.append(ols_bt[['quarter_end','ls_ret','cum_ls','model']])
dfs.append(linreg[['quarter_end','ls_ret','cum_ls','model']])
dfs.append(gbr[['quarter_end','ls_ret','cum_ls','model']])

all_bt = pd.concat(dfs, ignore_index=True)

# Summary stats
summary = []
for model, grp in all_bt.groupby('model'):
    final_ret = grp['cum_ls'].iloc[-1]
    avg_q = grp['ls_ret'].mean()
    vol_q = grp['ls_ret'].std()
    sharpe = avg_q / vol_q if vol_q > 0 else np.nan
    summary.append({
        'Model': model,
        'Final cumulative return': round(final_ret, 3),
        'Avg quarterly return': round(avg_q, 3),
        'Volatility': round(vol_q, 3),
        'Sharpe ratio': round(sharpe, 2)
    })

summary_df = pd.DataFrame(summary).set_index('Model')
print(summary_df)

# Save
outpath = PROJECT_ROOT/'data'/'backtest_summary.csv'
summary_df.to_csv(outpath)
print("\nSaved summary table:", outpath)


                   Final cumulative return  Avg quarterly return  Volatility  \
Model                                                                          
Gradient Boosting                    1.086                 0.011       0.051   
Linear Regression                    0.855                -0.016       0.085   
OLS                                  0.855                -0.016       0.085   

                   Sharpe ratio  
Model                            
Gradient Boosting          0.22  
Linear Regression         -0.19  
OLS                       -0.19  

Saved summary table: /content/drive/MyDrive/alpha_project/data/backtest_summary.csv
